# 자유음성 데이터

변환한 CSV 파일을 이용해서 전처리하는 과정입니다.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import re

In [2]:
dataSetPath = '/DATA/code/classification/data/train_filterd_output.csv'
saveSetPath = '/DATA/code/classification/data'

splitRate = 0.1 # valid

In [3]:
df = pd.read_csv(dataSetPath)
df

,file_name,gender,age,record_time,text
0,일반남여_일반통합01_F_syj050523_22_수도권_실내_00068.wav,1,22,3.60,처음 전화가 오면 보통 뭐라고 말해
1,일반남여_일반통합01_F_syj050523_22_수도권_실내_04064.wav,1,22,3.12,아마도
2,일반남여_일반통합01_F_syj050523_22_수도권_실내_00842.wav,1,22,5.16,설탕이 들어간 간식이 주는 가짜 행복은 일시적이라네요
3,일반남여_일반통합01_F_syj050523_22_수도권_실내_02265.wav,1,22,3.68,다음번 회식 때 기대할게
4,일반남여_일반통합01_F_syj050523_22_수도권_실내_02307.wav,1,22,2.12,맞아.
...,...,...,...,...,...
1037469,일반남여_일반통합13_M_1577813879_31_수도권_실내_19816.wav,0,31,5.72,여행 떠나는 날 나한테 전화해서 자랑한 거 기억 안 나요
1037470,일반남여_일반통합13_M_1577813879_31_수도권_실내_19613.wav,0,31,5.72,다음주 토요일 박교수님 큰아들 결혼식 있는 거 알고 있지
1037471,일반남여_일반통합13_M_1577813879_31_수도권_실내_19615.wav,0,31,5.12,그러게 말이야 시간이 어떻게 가는지 모르고 사는 것 같네
1037472,일반남여_일반통합13_M_1577813879_31_수도권_실내_20201.wav,0,31,8.02,저도 처음에는 그런 줄 알았는데 내가 족발 이러면 화를 내더라고요 맨날 그런 것만 ...


In [4]:
# 이상한 wav 파일 제거

no_audio_list = ['일반남여_일반통합12_M_1571527490_28_충청_실내_17186.wav', '일반남여_일반통합13_M_zzqkfkazz_25_기타_실내_19992.wav']

no_audio_index = df[(df['file_name'] == no_audio_list[0]) | (df['file_name'] == no_audio_list[1])]
no_audio_index = list(no_audio_index.index)

df = df.drop(no_audio_index, axis=0)

In [5]:
# 순서 섞기
df_shuffled = df.sample(frac=1).reset_index(drop=True)
df_shuffled

,file_name,gender,age,record_time,text
0,일반남여_일반통합08_M_junany96528_26_기타_실내_11578.wav,0,26,3.11,얼마나 들까
1,일반남여_일반통합09_M_1572134067_32_수도권_실내_13536.wav,0,32,2.64,배변을 보기가 훨씬 수월해질테니.
2,일반남여_일반통합16_F_1571783559_36_충청_실내_24342.wav,1,36,4.35,잔소리는 지금 엄마가 하고 있음요
3,일반남여_일반통합12_M_1553848291_39_수도권_실내_16278.wav,0,39,4.86,근데 판소리는 소리가 좀 거칠고 투박한 것 같아 강해 한 마디로
4,일반남여_일반통합10_M_1540357312_23_수도권_실내_12671.wav,0,23,5.55,내가 인상이 강하다는 얘기를 하도 많이 들어서 안경을 쓰면 좀 부드러운 인상으로 보...
...,...,...,...,...,...
1037467,일반남여_일반통합10_M_1524758851_26_수도권_실내_13389.wav,0,26,2.39,내가 주문할게 넌 뭐 먹을래
1037468,일반남여_일반통합11_M_ht8090_27_기타_실내_18444.wav,0,27,6.83,맞아 뭔가 대단한 걸 한건 아니지만 해냈다는 기분에 도취되지
1037469,일반남여_일반통합05_M_1541540881_38_수도권_실내_06862.wav,0,38,5.03,작년에도 걸렸는데 왜 올해 하셨어요
1037470,일반남여_일반통합05_M_1569774336_30_경상_실내_07003.wav,0,30,5.46,주방과 거실 등 주택의 일부를 공동으로 사용하는 거지


사람 수 조절

In [6]:
people = 75000 # 사람 수를 조절해서 데이터 길이를 조절합니다.
ftime = df_shuffled[df_shuffled['gender'] == 1][:people]['record_time'].sum() # 여성
mtime = df_shuffled[df_shuffled['gender'] == 0][:people]['record_time'].sum() # 남성

(ftime + mtime) / 3600

200.01828277777776

In [8]:
df_f = df_shuffled[df_shuffled['gender'] == 1][:people]
df_m = df_shuffled[df_shuffled['gender'] == 0][:people]

In [9]:
finaldf = pd.concat([df_f,df_m], axis=0).reset_index(drop=True)
finaldf = finaldf[['file_name','gender']]

In [10]:
# file_name에 경로수정
def modify_file_name(file_name):
    parts = file_name.split('_')
    new_path = f"/DATA2/자유대화 음성(일반남녀)/Training/{'_'.join(parts[:7])}/{file_name}"
    return new_path

# Apply the function to the file_name column
finaldf['file_name'] = finaldf['file_name'].apply(modify_file_name)
finaldf

,file_name,gender
0,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합05_F_fa...,1
1,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합11_F_15...,1
2,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합01_F_wl...,1
3,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합07_F_15...,1
4,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합07_F_15...,1
...,...,...
149995,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합10_M_wl...,0
149996,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합05_M_15...,0
149997,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합05_M_ke...,0
149998,/DATA2/자유대화 음성(일반남녀)/Training/일반남여_일반통합01_M_je...,0


In [12]:
train, valid = train_test_split(finaldf, test_size=splitRate)

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

train.to_csv(os.path.join(saveSetPath, 'train.csv'), index=False)
valid.to_csv(os.path.join(saveSetPath, 'test.csv'), index=False)